In [1]:
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate, FusekiQuery
from SPARQLWrapper import SPARQLWrapper, JSON
from api.src.schemas.bibframe.work import Work_Schema

In [2]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

Title

In [3]:
obj = {
  "content": "Text",
  "mainTitle": "Conjecturas e refutações",
  "subtitle": "refutações",
  "primaryContributionAgent": "Inácio Oliveira",
  "primaryContributionUri": "n80032184",
  "primaryContributionRole": "aut",
  "subjects": [
    {
    "label": "Methodology",
     "uri": "https://bibliokeia.com/authorities/subjects/sh85103704"
    }, 
    {
    "label": "Methodology",
     "uri": "https://bibliokeia.com/authorities/subjects/sh85103704"
    }
  ],
  "language": "Portugues",
  "languageCode": "por",
  "cdd": "584.6",
  "cutter": "A586w"
}

request = Work_Schema(**obj)
request


Work_Schema(ID=None, content='Text', mainTitle='Conjecturas e refutações', subtitle='refutações', variantTitle=None, primaryContributionAgent='Inácio Oliveira', primaryContributionUri='n80032184', primaryContributionRole='aut', subjects=[Subject(label='Methodology', uri='https://bibliokeia.com/authorities/subjects/sh85103704'), Subject(label='Methodology', uri='https://bibliokeia.com/authorities/subjects/sh85103704')], language='Portugues', languageCode='por', cdd='584.6', cutter='A586w', serie=None, serieURI=None)

In [3]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [23]:
r = acervoQuery.run_sparql("""PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
ASK { 
  graph work:bk-2
  {work:bk-2 rdf:type bf:Texto}}""")
r.convert()

{'head': {}, 'boolean': False}

In [99]:
def Edit(contentType, bkID):

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>"""
    where = "{work:"+bkID+" rdf:type ?o}"
                
    select = prefix+"SELECT * { graph work:bk-2 {work:"+bkID+" rdf:type ?o} }"

    response = acervoQuery.run_sparql(select)
    response = response.convert()
    bindings = response['results']['bindings']

    for binding in bindings:
        uri = binding['o']['value']
        content = uri.split("/")[-1]
        if content != 'Work' and content != contentType:
            where = "{ work:"+bkID+" rdf:type bf:"+content+" }"
            up = prefix+"WITH work:"+bkID+"""
            DELETE """+where+"""
            INSERT { work:"""+bkID+" rdf:type bf:"+contentType+""" }
            WHERE """+where
            acervoUpdate.run_sparql(up)


Edit("Text", "bk-2")

TITLE

In [7]:
def EditTitle(mainTitle, bkID):

    acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
    acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""
    # Main Title
    ask = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:mainTitle '"""+mainTitle+"' }}"""
    response = acervoQuery.run_sparql(ask)
    response = response.convert()

    if not response['boolean']:  
        up = prefix+"WITH work:"+bkID+"""
                    DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle '"""+mainTitle+"""' }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle ?t }"""

        acervoUpdate.run_sparql(up)

        label = prefix+"WITH work:"+bkID+"""
                    DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label '"""+mainTitle+"""' }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label ?t }"""
        acervoUpdate.run_sparql(label)
    
    # Subtitle
    askSubtitle = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle '"""+mainTitle+"' }}"""
    
    askResponse = acervoQuery.run_sparql(askSubtitle)
    askResponse = askResponse.convert()
    if not askResponse['boolean']: 
        print('PRECISA INCLUIR SUBTITULO')
    else:
        print('NÃO PRECISA FAZER NADA')



EditTitle("TITULO_5", "bk-2")

PRECISA INCLUIR SUBTITULO


In [42]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [65]:
def EditSubtitle(subtitle, title,  bkID):    

    acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
    acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""
    
    subtitleSame = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle '"""+subtitle+"' }}"
    
    askSubtitleSame = acervoQuery.run_sparql(subtitleSame)
    askSubtitleSame = askSubtitleSame.convert()

    if not askSubtitleSame['boolean']:
        subtitleExist = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle ?subtitle }}"""
        askSubtitleExist = acervoQuery.run_sparql(subtitleExist)
        askSubtitleExist = askSubtitleExist.convert()

        if askSubtitleExist['boolean']:
            up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                    ?o rdf:type bf:Title .
                        ?o bf:mainTitle '"""+title+"""' .
                        ?o bf:subtitle '"""+subtitle+"""'
                          }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }"""
            acervoUpdate.run_sparql(up)
        else:
            up = prefix+"INSERT DATA { GRAPH work:"+bkID+" { work:"+bkID+"""  bf:title [
                        rdf:type bf:Title;
                        bf:mainTitle '"""+title+"""' ;
                        bf:subtitle '"""+subtitle+"""' ] . } }"""
            acervoUpdate.run_sparql(up)


EditSubtitle("Subtitulo", "Titulo", "bk-2")  

SUBTITULO NÃO EXISTE


DELETE

In [64]:
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }
                WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }"""

response = acervoUpdate.run_sparql(up)
response = response.convert()
response

{'statusCode': 200, 'message': 'Update succeeded'}

In [21]:
bkID = 'bk-1'
request.primaryContributionRole

'Autor'

AUTOR

In [24]:
askRole = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+""" bf:contribution ?o .
                        ?o bf:role <http://id.loc.gov/vocabulary/relators/"""+request.primaryContributionRole+"""> }}"""
response = acervoQuery.run_sparql(askRole)
response = response.convert()
response

{'head': {}, 'boolean': False}

Subjects

In [3]:
obj = {
  "content": "Text",
  "mainTitle": "MUDADDO",
  "subtitle": "refutações",
  "primaryContributionAgent": "Inácio Oliveira",
  "primaryContributionUri": "n80032184",
  "primaryContributionRole": "aut",
  "subjects": [
    {
    "label": "Methodology",
     "uri": "https://bibliokeia.com/authorities/subjects/sh85103704"
    }, 
    {
    "label": "Ciência",
     "uri": "https://bibliokeia.com/authorities/subjects/n80002329"
    },
    {
    "label": "Manga",
     "uri": "https://bibliokeia.com/authorities/subjects/sh85084414"
    }    
  ],
  "language": "Portugues",
  "languageCode": "por",
  "cdd": "584.6",
  "cutter": "A586w"
}

request = Work_Schema(**obj)
request

Work_Schema(ID=None, content='text', mainTitle='Conjecturas e refutações', subtitle='refutações', variantTitle=None, primaryContributionAgent='Inácio Oliveira', primaryContributionUri='n80032184', primaryContributionRole='Autor', subjects=[Subject(label='Methodology', uri='https://bibliokeia.com/authorities/subjects/sh85103704'), Subject(label='Ciência', uri='https://bibliokeia.com/authorities/subjects/n80002329')], language='Portugues', languageCode='por', cdd='584.6', cutter='A586w', serie=None, serieURI=None)

In [6]:
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

In [5]:

listSubjects = list()
for subject in request.subjects:
    askSubject = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+""" bf:subject <"""+subject.uri+"> . }}"
    response = acervoQuery.run_sparql(askSubject)
    response = response.convert()
    listSubjects.append(response['boolean'])
    print(subject.uri)

https://bibliokeia.com/authorities/subjects/sh85103704
https://bibliokeia.com/authorities/subjects/n80002329


In [13]:
slist = list()
for subject in request.subjects:
    s = f" bf:subject <{subject.uri}> "
    slist.append(s)
sub = "; ".join(slist)
print(slist)

[' bf:subject <https://bibliokeia.com/authorities/subjects/sh85103704> ', ' bf:subject <https://bibliokeia.com/authorities/subjects/n80002329> ']


In [16]:
up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:subject ?o }
                INSERT {work:"""+bkID+sub+""" }
                WHERE {work:"""+bkID+""" bf:subject ?o }"""
acervoUpdate.run_sparql(up)

Language

In [25]:
request.languageCode

'por'

In [27]:
askLanguage = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+" bf:language <http://id.loc.gov/vocabulary/languages/"+request.languageCode+"""> }} """
response = acervoQuery.run_sparql(askLanguage)
response = response.convert()
response

{'head': {}, 'boolean': True}

Classification

In [8]:
bkID = 'bk-1'

In [9]:
askClassification = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+""" bf:classification ?obj .
                        ?obj bf:classificationPortion """+request.cdd+" }} "
response = acervoQuery.run_sparql(askClassification)
response = response.convert()
response

{'head': {}, 'boolean': False}

In [10]:
request.cutter

'A586w'

In [15]:
askCutter = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+""" bf:classification ?obj .
                        ?obj bf:itemPortion '"""+request.cutter+"' }} "
responseCutter = acervoQuery.run_sparql(askCutter)
responseCutter = responseCutter.convert()
responseCutter

{'head': {}, 'boolean': True}

In [39]:
up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:classification ?obj . 
                   ?obj ?p ?o }
                WHERE {work:"""+bkID+""" bf:classification ?obj .
                   ?obj ?p ?o }"""
acervoUpdate.run_sparql(up)

In [16]:
up = prefix+"WITH work:"+bkID+"""
               DELETE {work:"""+bkID+""" bf:classification ?obj . 
                   ?obj ?p ?o }
                INSERT {work:"""+bkID+""" bf:classification ?obj . 
                  ?obj rdf:type bf:ClassificationDdc .
                  ?obj bf:classificationPortion """+request.cdd+""" . 
                  ?obj bf:itemPortion '"""+request.cutter+"""'  }
                WHERE {work:"""+bkID+""" bf:classification ?obj .
                   ?obj ?p ?o }"""
acervoUpdate.run_sparql(up)